# Adversarial Search: Playing Connect 4


## Instructions

Total Points: Undegraduates 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play Connect 4:

> "Connect 4 is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs." (see [Connect Four on Wikipedia](https://en.wikipedia.org/wiki/Connect_Four))

## Task 1: Defining the Search Problem [1 point]

Define the components of the search problem:

* Initial state: an empty board.
* Actions: unfilled columns, {0-6} at game start.
* Transition model: When a player drops a piece, it falls straight down to the lowest available space in the column.
* Goal state: Four of the player's pieces are lined up vertically, horizontally, or diagonally.

Each of the 42 squares can have three values (empty, x, or o). However,
* pieces cannot float above empty squares
* the # of x's is always within one of the # of o's
* both players can't have 4-in-a-row at once, etc.

So, the search space has significantly less than this number of states:

In [1]:
print(3**42)
print('{:.1e}'.format(3**42))

109418989131512359209
1.1e+20


## Task 2: Game Environment and Random Agent [2 point]

Implement an agent that plays randomly and let two random agents play against each other 1000 times. How often does each player win? Is the result expected?

In [29]:
"""sources"""
# winning_move() and its kernels -> https://stackoverflow.com/a/63991845

"""imports"""
from scipy.signal import convolve2d
import sys
import numpy as np

"""rng seeding"""
np.random.seed(0)

"""constants"""
DEBUG = 2
HEIGHT = 6
WIDTH = 7

"""kernels for winning_move()"""
horizontal_kernel = np.array([[ 1, 1, 1, 1]])
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(4, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

def empty_board(shape=(HEIGHT, WIDTH)):
    return np.full(shape=shape, fill_value=' ')

def winning_move(board, player):
    for kernel in detection_kernels:
        if (convolve2d(board == player, kernel, mode="valid") == 4).any():
            return True
    return False

def check_win(board):
    """check the board and return one of x, o, d (draw), or n (for next move)"""
    
    # check for win
    if(winning_move(board, 'x')): return 'x'
    if(winning_move(board, 'o')): return 'o'
    
    # check for draw
    if(np.sum(board == ' ') < 1): return 'd'
    
    return 'n'

def get_actions(board):
    """Returns non-full columns as a vector of indices"""
    top_row = board[0, 0:7]
    actions = np.where(top_row == ' ')[0].tolist()
    if(DEBUG): print(actions)
    return actions

def other(player): 
    if player == 'x': return 'o'
    else: return 'x'

def result(state, player, action):
    """Add move to the board"""
    for i in range(HEIGHT):
        if(state[HEIGHT-i-1][action] == ' '):
            state[HEIGHT-i-1][action] = player
            return state
    sys.exit('error -> column full')

def utility(state, player = 'x'):
    """check if a state is terminal and return the utility if it is. None means not a terminal mode."""
    goal = check_win(state)        
    if goal == player: return +1 
    if goal == 'd': return 0  
    if goal == other(player): return -1  # loss is failure
    return None # continue

def is_terminal(state): 
    """check is a state is a terminal state"""
    return check_win(state) != "n"

def random_player(board, player = None):
    """Simple player that chooses a random unfilled column.
       Argument 'player' is unused."""
    action = np.random.choice(get_actions(board), size=None, replace=False)
    if(DEBUG): print(action)
    return action

def switch_player(player, x, o):
    if player == 'x':
        return 'o', o
    else:
        return 'x', x

def play(x, o, N = 1):
    results = {'x': 0, 'o': 0, 'd': 0}
    for i in range(N):
        board = empty_board()
        player, fun = 'x', x
        
        while True:
            a = fun(board, player)
            board = result(board, player, a)
            
            win = check_win(board)
            if win != 'n':
                results[win] += 1
                break
            
            player, fun = switch_player(player, x, o)   
    return results

    
if(DEBUG>1):
    """Test check_win"""
    board = empty_board()
    for i in range(4): result(board, 'x', 0)
    print(board)
    print('Win? ' + check_win(board))
    print()
    board = empty_board()
    for i in range(4): result(board, 'o', 0)
    print(board)
    print('Win? ' + check_win(board))
    print()
    board = empty_board()
    print(board)
    print('Win? ' + check_win(board))
    print()
    
    """Test utility"""
    print(utility(np.full(shape=[HEIGHT, WIDTH], fill_value='x')))
    print(utility(np.full(shape=[HEIGHT, WIDTH], fill_value='o')))
    print(utility(empty_board()))
    print()
    
    """Test is_terminal"""
    print(is_terminal(np.full(shape=[HEIGHT, WIDTH], fill_value='x')))
    print(is_terminal(np.full(shape=[HEIGHT, WIDTH], fill_value='o')))
    print(is_terminal(empty_board()))
    
if(DEBUG<2):
    %timeit -n 1 -r 1 display(play(random_player, random_player, N = 1000))

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' ' ' ' ' ' ' ' ']]
Win? x

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['o' ' ' ' ' ' ' ' ' ' ' ' ']
 ['o' ' ' ' ' ' ' ' ' ' ' ' ']
 ['o' ' ' ' ' ' ' ' ' ' ' ' ']
 ['o' ' ' ' ' ' ' ' ' ' ' ' ']]
Win? o

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']]
Win? n

1
-1
None

True
True
False


* Player 'x' won 560 times and player 'o' won 437 times.
* I expected this result, because player 'x' always goes first. This makes it more likely for player 'x' to win, because they are the first player that is able to win the game (on their fourth turn) and always has more or equal the number of turns taken compared to player 'o'.

## Task 3: Minimax Search with Alpha-Beta Pruning [4 points]

### Implement the search starting from a given board and specifying the player.

Experiment with some manually created boards (at least 5) to check if the agent spots winning opportunities.

In [4]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [5]:
# Your code/ answer goes here.

### Move ordering

Describe and implement a simple move ordering strategy. How does this strategy influence the time it takes to 
make a move?

In [6]:
# Your code/ answer goes here.

### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [7]:
# Your code/ answer goes here.

## Task 4: Heuristic Alpha-Beta Tree Search [3 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [8]:
# Your code/ answer goes here.

### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [9]:
# Your code/ answer goes here.

Experiment with the same manually created boards as above to check if the agent spots wining opportunities.

In [10]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [11]:
# Your code/ answer goes here.

### Forward Pruning

Add forward pruning to the cutoff search where you do not consider moves that have a low evaluation value after a shallow search 
(way smaller than the cuttoff value).

In [12]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [13]:
# Your code/ answer goes here.

### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function or different forward pruning) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [14]:
# Your code/ answer goes here.

## Challenge task [+ 1 bonus point]

Find another student and let your best agent play against the other student's best player. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [1 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

### Pure Monte Carlos Search

Implement Pure Monte Carlo Search and investigate how this search performs on the test boards that you have used above. 

In [15]:
# Your code/ answer goes here.

### Best First Move

How would you determine what the best first move is? You can use Pure Monte Carlo Search or any algorithms 
that you have implemented above.

In [16]:
# Your code/ answer goes here.